# Step 1. Import data from the webscraping

In [18]:
import pandas as pd
import glob
import os

# Get the parent directory path
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

# Find all files in the parent directory matching the webscrape pattern
csv_pattern = os.path.join(parent_dir, "webscrape_*.csv")
csv_files = glob.glob(csv_pattern)

# Store column headers for each file
all_headers = {}
consistent = True
first_file = None

print(f"Found {len(csv_files)} webscrape CSV files")

# Loop through each file and get headers
for file in csv_files:
    filename = os.path.basename(file)
    
    # Just read the headers without loading entire file
    headers = pd.read_csv(file, nrows=0).columns.tolist()
    all_headers[filename] = headers
    
    # Store first file's headers as reference
    if first_file is None:
        first_file = filename
        reference_headers = headers
        print(f"Reference headers from {first_file}: {reference_headers}")
    # Compare current file with reference
    elif headers != reference_headers:
        consistent = False
        print(f"\nMISMATCH in {filename}:")
        
        # Find and show differences
        missing = set(reference_headers) - set(headers)
        extra = set(headers) - set(reference_headers)
        
        if missing:
            print(f"  Missing columns: {missing}")
        if extra:
            print(f"  Extra columns: {extra}")

# Print final result
if consistent:
    print("\nAll files have identical column headers")
else:
    print("\nWarning: Column headers differ between files")

Found 66 webscrape CSV files
Reference headers from webscrape_2023-08-15_09-20-02.csv: ['title', 'company', 'location', 'salary', 'description']

All files have identical column headers


# Step 2. Add date and date-times so we can identify where each job came from

In [20]:
# Create DataFrames with dates extracted from filenames
import pandas as pd
import re
from datetime import datetime

# Dictionary to store all DataFrames
dataframes = {}

# Regular expression to extract date and time from filename
date_pattern = r'webscrape_(\d{4}-\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})\.csv'

# Process each CSV file
for file in csv_files:
    filename = os.path.basename(file)
    
    # Extract date using regex
    match = re.search(date_pattern, filename)
    if match:
        date_str = match.group(1)  # YYYY-MM-DD
        time_str = match.group(2).replace('-', ':')  # Convert to HH:MM:SS
        datetime_str = f"{date_str} {time_str}"
        
        # Parse to datetime object
        file_datetime = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')
        
        # Read the CSV
        df = pd.read_csv(file)
        
        # Add date columns
        df['scrape_date'] = file_datetime.date()
        df['scrape_datetime'] = file_datetime
        
        # Store in dictionary
        dataframes[filename] = df
        
        print(f"Loaded {filename} with {len(df)} rows, scraped on {file_datetime}")
    else:
        print(f"Warning: Couldn't extract date from {filename}")

print(f"\nProcessed {len(dataframes)} files")

# Example to verify date columns were added correctly
if dataframes:
    first_key = list(dataframes.keys())[0]
    print(f"\nExample from {first_key}:")
    print(dataframes[first_key][['scrape_date', 'scrape_datetime']].head(2))

Loaded webscrape_2023-08-15_09-20-02.csv with 3923 rows, scraped on 2023-08-15 09:20:02
Loaded webscrape_2023-08-23_09-55-48.csv with 4151 rows, scraped on 2023-08-23 09:55:48
Loaded webscrape_2023-08-30_10-13-36.csv with 3897 rows, scraped on 2023-08-30 10:13:36
Loaded webscrape_2023-09-07_09-32-27.csv with 4266 rows, scraped on 2023-09-07 09:32:27
Loaded webscrape_2023-09-27_11-14-05.csv with 3458 rows, scraped on 2023-09-27 11:14:05
Loaded webscrape_2023-10-04_09-34-49.csv with 3712 rows, scraped on 2023-10-04 09:34:49
Loaded webscrape_2023-10-11_07-59-43.csv with 3362 rows, scraped on 2023-10-11 07:59:43
Loaded webscrape_2023-10-18_09-22-00.csv with 3899 rows, scraped on 2023-10-18 09:22:00
Loaded webscrape_2023-11-01_08-31-18.csv with 4629 rows, scraped on 2023-11-01 08:31:18
Loaded webscrape_2023-11-08_10-36-53.csv with 1859 rows, scraped on 2023-11-08 10:36:53
Loaded webscrape_2023-11-22_10-11-56.csv with 2762 rows, scraped on 2023-11-22 10:11:56
Loaded webscrape_2023-11-29_10-0

# Step 3. Merge the webscrape CSVs

In [21]:
# Merge all DataFrames into one combined dataset
import pandas as pd

# Check if we have DataFrames to merge
if not dataframes:
    print("No DataFrames to merge!")
else:
    # Combine all DataFrames from the dictionary
    combined_df = pd.concat(dataframes.values(), ignore_index=True)
    
    # Basic info about the combined dataset
    print(f"Combined dataset created successfully!")
    print(f"Total rows: {len(combined_df):,}")
    print(f"Total columns: {len(combined_df.columns)}")
    
    # Show unique scrape dates to verify we have data from different days
    unique_dates = combined_df['scrape_date'].unique()
    print(f"\nData includes {len(unique_dates)} unique dates:")
    for date in sorted(unique_dates):
        count = len(combined_df[combined_df['scrape_date'] == date])
        print(f"  {date}: {count:,} rows")
    
    # Preview of the combined data
    print("\nPreview of combined dataset:")
    # Show date columns first, then a few others
    preview_cols = ['scrape_date', 'scrape_datetime'] 
    # Add a few more columns for preview (adjust as needed)
    other_cols = [col for col in combined_df.columns 
                 if col not in ['scrape_date', 'scrape_datetime']][:3]
    preview_cols.extend(other_cols)
    
    print(combined_df[preview_cols].head())

Combined dataset created successfully!
Total rows: 227,210
Total columns: 7

Data includes 66 unique dates:
  2023-08-15: 3,923 rows
  2023-08-23: 4,151 rows
  2023-08-30: 3,897 rows
  2023-09-07: 4,266 rows
  2023-09-27: 3,458 rows
  2023-10-04: 3,712 rows
  2023-10-11: 3,362 rows
  2023-10-18: 3,899 rows
  2023-11-01: 4,629 rows
  2023-11-08: 1,859 rows
  2023-11-22: 2,762 rows
  2023-11-29: 2,931 rows
  2023-12-06: 3,958 rows
  2023-12-13: 4,141 rows
  2023-12-20: 3,775 rows
  2024-01-03: 3,193 rows
  2024-01-10: 4,081 rows
  2024-01-17: 3,630 rows
  2024-01-31: 3,239 rows
  2024-02-07: 3,268 rows
  2024-02-14: 3,427 rows
  2024-02-21: 3,644 rows
  2024-03-13: 3,050 rows
  2024-03-20: 3,491 rows
  2024-03-27: 2,945 rows
  2024-04-10: 3,504 rows
  2024-04-17: 4,330 rows
  2024-04-24: 3,500 rows
  2024-05-01: 2,978 rows
  2024-05-08: 3,199 rows
  2024-05-15: 2,871 rows
  2024-05-22: 2,951 rows
  2024-05-29: 2,248 rows
  2024-06-05: 2,983 rows
  2024-06-12: 2,814 rows
  2024-06-19: 2,5

# Step 4. Tidying data.

## 4.1. Removing rows that aren't jobs.

### 4.1.1 Where 'illustration of bank notes' in job title

In [22]:
# Remove rows where title contains "Illustration of banknotes" with flexible matching

# First check if title column exists
if "title" not in combined_df.columns:
    print("Warning: 'title' column not found in the dataset.")
else:
    # Display some titles to understand what we're working with
    print("Sample of titles in the dataset:")
    sample_titles = combined_df["title"].dropna().sample(min(5, len(combined_df))).tolist()
    for title in sample_titles:
        print(f"  - {title}")
    
    # Count original rows
    original_count = len(combined_df)
    
    # Look at potential matches more flexibly
    potential_matches = combined_df[combined_df["title"].str.contains("Illustration of banknote", case=False, na=False)]
    print(f"\nFound {len(potential_matches)} potential matches with 'Illustration of banknote'")
    
    if len(potential_matches) > 0:
        print("\nSample of matching titles:")
        for title in potential_matches["title"].unique()[:5]:
            print(f"  - {title}")
    
    # Remove rows with flexible matching
    filtered_df = combined_df[~combined_df["title"].str.contains("Illustration of banknote", case=False, na=False)]
    
    # Print results
    print(f"\nOriginal dataset: {original_count:,} rows")
    print(f"Removed: {original_count - len(filtered_df):,} rows")
    print(f"Filtered dataset: {len(filtered_df):,} rows")
    
    # Replace original with filtered data
    combined_df = filtered_df.copy()

Sample of titles in the dataset:
  -                     Wellbeing Personal Trainer
  -                     Pharmacy Assistant
  -                     Dietetics Assistant
  -                     Activities Assistant
  -                     Deputy Housekeeping Manager

Found 4482 potential matches with 'Illustration of banknote'

Sample of matching titles:
  -                 Illustration of banknotes
  - 
                Illustration of banknotes

Original dataset: 227,210 rows
Removed: 4,482 rows
Filtered dataset: 222,728 rows
